<a href="https://colab.research.google.com/github/vivek6311/Artificial-Intelligence-with-Python/blob/master/Fuzzy_Logic_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
pip install scikit-fuzzy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 994.0/994.0 KB 46.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-fuzzy: filename=scikit_fuzzy-0.4.2-py3-none-any.whl size=894086 sha256=218a750b977cd91f5cb629d1b31259bb55dc35174be4cd50ee4a0a644ad027dc
  Stored in directory: /root/.cache/pip/wheels/32/2c/a1/a90a7d7dd8448ec029f298a61f3490275e99b17aa348be675c
Successfully built scikit-fuzzy


In [5]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Step 1: Gather the necessary data
# Assume that we have already obtained the data and stored it in the following arrays
weather = np.array([25, 28, 30, 27, 26, 24, 23, 22, 21, 20]) # average temperature in Celsius
soil = np.array([0.2, 0.3, 0.5, 0.6, 0.7, 0.8, 0.9, 0.8, 0.7, 0.6]) # soil moisture content
seed = np.array([0.8, 0.9, 0.7, 0.6, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]) # seed quality
management = np.array([0.3, 0.5, 0.7, 0.6, 0.8, 0.9, 0.7, 0.6, 0.4, 0.5]) # management practices
yield_actual = np.array([5, 6, 7, 8, 9, 8, 7, 6, 5, 4]) # actual crop yield

# Step 2: Preprocess the data
X = np.column_stack((weather, soil, seed, management))
y = yield_actual
n = len(y)
n_train = int(0.7 * n) # use 70% of the data for training

X_train = X[:n_train]
y_train = y[:n_train]
X_test = X[n_train:]
y_test = y[n_train:]

# Step 3: Apply fuzzy logic
weather_level = ctrl.Antecedent(np.arange(20, 31, 1), 'weather')
soil_level = ctrl.Antecedent(np.arange(0, 1.1, 0.1), 'soil')
seed_level = ctrl.Antecedent(np.arange(0.5, 1.1, 0.1), 'seed')
management_level = ctrl.Antecedent(np.arange(0, 1.1, 0.1), 'management')
yield_level = ctrl.Consequent(np.arange(0, 11, 1), 'yield')

weather_level['low'] = fuzz.trimf(weather_level.universe, [20, 22, 25])
weather_level['medium'] = fuzz.trimf(weather_level.universe, [23, 26, 29])
weather_level['high'] = fuzz.trimf(weather_level.universe, [27, 30, 31])

soil_level['dry'] = fuzz.trimf(soil_level.universe, [0, 0, 0.5])
soil_level['moderate'] = fuzz.trimf(soil_level.universe, [0, 0.5, 1])
soil_level['wet'] = fuzz.trimf(soil_level.universe, [0.5, 1, 1])

seed_level['poor'] = fuzz.trimf(seed_level.universe, [0.5, 0.5, 0.8])
seed_level['average'] = fuzz.trimf(seed_level.universe, [0.6, 0.6, 0.7])


In [6]:
seed_level['good'] = fuzz.trimf(seed_level.universe, [0.7, 0.9, 1])

management_level['poor'] = fuzz.trimf(management_level.universe, [0, 0, 0.5])
management_level['average'] = fuzz.trimf(management_level.universe, [0.4, 0.5, 0.6])
management_level['good'] = fuzz.trimf(management_level.universe, [0.5, 1, 1])

yield_level['low'] = fuzz.trimf(yield_level.universe, [0, 0, 5])
yield_level['medium'] = fuzz.trimf(yield_level.universe, [3, 5, 7])
yield_level['high'] = fuzz.trimf(yield_level.universe, [5, 10, 10])

rule1 = ctrl.Rule(weather_level['low'] | soil_level['dry'], yield_level['low'])
rule2 = ctrl.Rule(weather_level['high'] | soil_level['wet'] | seed_level['good'] | management_level['good'], yield_level['high'])
rule3 = ctrl.Rule(weather_level['medium'] & (seed_level['average'] | management_level['average']), yield_level['medium'])

yield_ctrl = ctrl.ControlSystem([rule1, rule2, rule3])
yield_prediction = ctrl.ControlSystemSimulation(yield_ctrl)

# Step 4: Train the regression model
reg_model = LinearRegression()
reg_model.fit(X_train, y_train)

# Step 5: Evaluate the model performance
y_pred = reg_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

# Step 6: Make yield predictions for the test data using fuzzy logic and regression model
yield_predicted = []
for i in range(len(X_test)):
    weather_value = X_test[i][0]
    soil_value = X_test[i][1]
    seed_value = X_test[i][2]
    management_value = X_test[i][3]

    yield_prediction.input['weather'] = weather_value
    yield_prediction.input['soil'] = soil_value
    yield_prediction.input['seed'] = seed_value
    yield_prediction.input['management'] = management_value

    yield_prediction.compute()

    yield_predicted.append(yield_prediction.output['yield'])

yield_predicted = np.array(yield_predicted)

# Step 7: Compare the predicted yield with the actual yield
mse_fuzzy = mean_squared_error(y_test, yield_predicted)
r2_fuzzy = r2_score(y_test, yield_predicted)

print(f"Mean Squared Error (fuzzy logic): {mse_fuzzy}")
print(f"R-squared (fuzzy logic): {r2_fuzzy}")


Mean Squared Error: 0.02173303198108245
R-squared: 0.9674004520283763
Mean Squared Error (fuzzy logic): 5.414353281797261
R-squared (fuzzy logic): -7.121529922695892
